In [0]:
!pip3 install tensorflow==1.13.1
import tensorflow as tf
print(tf.__version__)

In [2]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 3.5MB/s 

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [3]:
import numpy as np
import pandas as pd
import os
os.environ['TF_KERAS'] = '1'
import sys
import random
#import keras
import tensorflow as tf
import json
UPLOAD_TIME = '2019_05_30' #@param {type:"string"}
BERT_MODEL = 'wwm_uncased_L-24_H-1024_A-16' #@param {type:"string"}
download_url = 'https://storage.googleapis.com/bert_models/{}/{}.zip'.format(UPLOAD_TIME,BERT_MODEL)
zip_path = '{}.zip'.format(BERT_MODEL)
! test -d $BERT_MODEL || (wget $download_url && unzip $zip_path)
BERT_PRETRAINED_DIR = os.path.realpath(BERT_MODEL)
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py
import tokenization  #Actually keras_bert contains tokenization part, here just for convenience

--2019-07-29 10:26:10--  https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.196.128, 2607:f8b0:4001:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.196.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1248381879 (1.2G) [application/zip]
Saving to: ‘wwm_uncased_L-24_H-1024_A-16.zip’

wwm_uncased_L-24_H- 100%[===================>]   1.16G  72.4MB/s    in 13s     

2019-07-29 10:26:24 (92.1 MB/s) - ‘wwm_uncased_L-24_H-1024_A-16.zip’ saved [1248381879/1248381879]

Archive:  wwm_uncased_L-24_H-1024_A-16.zip
   creating: wwm_uncased_L-24_H-1024_A-16/
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.meta  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001  
  inflating: wwm_uncased_L-24_H-1024_A-16/vocab.txt  
  inflating: wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt.index  
  inflating: wwm_unc

In [4]:
! pip install keras-bert -q

In [5]:
def loadfrommydrive():  
  link = 'https://drive.google.com/open?id=1Uhre5CGrs7xdTAILCcseKPiJw073Czxw' # The shareable link
  fluff, id = link.split('=')
  print (id) # Verify that you have everything after '='
  downloaded1 = drive.CreateFile({'id':id}) 
  downloaded1.GetContentFile('train.csv.zip')

  link = 'https://drive.google.com/open?id=1neE7a2COjKXDA6LB7RFRb2lu5Ksy5frp' # The shareable link
  fluff, id = link.split('=')
  print (id) # Verify that you have everything after '='
  downloaded2 = drive.CreateFile({'id':id}) 
  downloaded2 = drive.CreateFile({'id':id}) 
  downloaded2.GetContentFile('test.csv.zip')  
  # Dataset is now stored in a Pandas Dataframe
  # Dataset is now stored in a Pandas Dataframe
  link = 'https://drive.google.com/open?id=1Aan7JSlxGPUEJgrO1cITMM0F2n6z_X9h' # The shareable link
  fluff, id = link.split('=')
  print (id) # Verify that you have everything after '='
  downloaded1 = drive.CreateFile({'id':id}) 
  downloaded1.GetContentFile('new7_train_input_220.npy')

loadfrommydrive()
! unzip train.csv.zip
! unzip test.csv.zip


1Uhre5CGrs7xdTAILCcseKPiJw073Czxw
1neE7a2COjKXDA6LB7RFRb2lu5Ksy5frp
1Aan7JSlxGPUEJgrO1cITMM0F2n6z_X9h
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


## Parameters and load training data

In [0]:
lr = 3e-5
weight_decay = 0.01
nb_epochs=1
bsz = 64
maxlen=220
sigma= 0.998
## Training data
## step parameter 
decay_steps = int(1.2*nb_epochs*1804874/bsz)
warmup_steps = int(0.05*decay_steps)

In [7]:
#from keras_bert.bert import get_model
from keras_bert import load_trained_model_from_checkpoint
print('begin_build')

config_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
checkpoint_file = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
model = load_trained_model_from_checkpoint(config_file, checkpoint_file, training=True,seq_len=maxlen)
#model.summary(line_length=120)

begin_build
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


## Load raw model

## Build classification model with adamwarmup

First folk the optimizer with excluding "bias" and "Norm" parameters from weight decay:

In [0]:
from tensorflow.keras import backend as K
from tensorflow.python import keras
def get_multiplier(p,lr_t):
  if 'Input' in p.name or 'Embedding' in p.name:
    return lr_t*(sigma**23)
  elif 'Extract' in p.name or 'real_output' in p.name or 'aux_output' in p.name:
    return lr_t*(sigma**0)
  else:
    for i in range(1,25):
      if '{}'.format(i) in p.name:
        return lr_t*(sigma**(24-i))
            
class AdamWarmup(keras.optimizers.Optimizer):
    def __init__(self, decay_steps, warmup_steps, min_lr=0.0,
                 lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=None, kernel_weight_decay=0., bias_weight_decay=0.,
                 amsgrad=False, **kwargs):
        super(AdamWarmup, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.decay_steps = K.variable(decay_steps, name='decay_steps')
            self.warmup_steps = K.variable(warmup_steps, name='warmup_steps')
            self.min_lr = K.variable(min_lr, name='min_lr')
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.kernel_weight_decay = K.variable(kernel_weight_decay, name='kernel_weight_decay')
            self.bias_weight_decay = K.variable(bias_weight_decay, name='bias_weight_decay')
        if epsilon is None:
            epsilon = K.epsilon()
        self.epsilon = epsilon
        self.initial_kernel_weight_decay = kernel_weight_decay
        self.initial_bias_weight_decay = bias_weight_decay
        self.amsgrad = amsgrad

    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]

        t = K.cast(self.iterations, K.floatx()) + 1

        lr = K.switch(
            t <= self.warmup_steps,
            self.lr * (t / self.warmup_steps),
            self.lr * (1.0 - K.minimum(t, self.decay_steps) / self.decay_steps),
        )

        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        if self.amsgrad:
            vhats = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        else:
            vhats = [K.zeros(1) for _ in params]
        self.weights = [self.iterations] + ms + vs + vhats

        for p, g, m, v, vhat in zip(params, grads, ms, vs, vhats):
            new_lr_t = get_multiplier(p,lr_t)#lr_t#
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            if self.amsgrad:
                vhat_t = K.maximum(vhat, v_t)
                p_t = m_t / (K.sqrt(vhat_t) + self.epsilon)
                self.updates.append(K.update(vhat, vhat_t))
            else:
                p_t = m_t / (K.sqrt(v_t) + self.epsilon)

            if 'bias' not in p.name and 'Norm' not in p.name:
                if self.initial_bias_weight_decay > 0.0:
                    p_t += self.bias_weight_decay * p
            else:
                if self.initial_kernel_weight_decay > 0.0:
                    p_t += self.kernel_weight_decay * p
            p_t = p - new_lr_t * p_t

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {
            'decay_steps': float(K.get_value(self.decay_steps)),
            'warmup_steps': float(K.get_value(self.warmup_steps)),
            'min_lr': float(K.get_value(self.min_lr)),
            'lr': float(K.get_value(self.lr)),
            'beta_1': float(K.get_value(self.beta_1)),
            'beta_2': float(K.get_value(self.beta_2)),
            'epsilon': self.epsilon,
            'kernel_weight_decay': float(K.get_value(self.kernel_weight_decay)),
            'bias_weight_decay': float(K.get_value(self.bias_weight_decay)),
            'amsgrad': self.amsgrad,
        }
        base_config = super(AdamWarmup, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

As the Extract layer extracts only the first token where "['CLS']" used to be, we just take the layer and connect to the single neuron output.

In [0]:
#weights and custom_loss ours before
from tensorflow.keras.losses import binary_crossentropy
def loadeverything():
  train_df = pd.read_csv('train.csv')
  train_df = train_df.sample(frac=1.0,random_state = 42+50000)
  AUX_COLUMNS = ['severe_toxicity','obscene','identity_attack','insult','threat','sexual_explicit']
  identity_columns = [
      'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
      'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
  coll = ['black','white','homosexual_gay_or_lesbian','muslim']

  weights = np.ones((len(train_df),)) / 4
  # Subgroup  identity_columns  > 0.5
  weights += (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
  # Background Positive, Subgroup Negative
  weights += (( (train_df['target'].values>=0.5).astype(bool).astype(np.int) +
     (train_df[identity_columns].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4
  # Background Negative, Subgroup Positive
  weights += (( (train_df['target'].values<0.5).astype(bool).astype(np.int) +
     (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 4

  weights += (( (train_df['target'].values>=0.5).astype(bool).astype(np.int) +
     (train_df[coll].fillna(0).values<0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) /8

  weights += (( (train_df['target'].values<0.5).astype(bool).astype(np.int) +
     (train_df[coll].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) / 8

  loss_weight = 1.0 / weights.mean()
  weights =weights.reshape(-1,1)

  y_aux = train_df[AUX_COLUMNS].values
  ###
  y = train_df['target'].values[:,None]
  y = np.hstack([y,weights])
  train_df = train_df.drop(['comment_text'], axis=1)
  train_idx = train_df.index.values
  print(train_idx)
  #np.random.shuffle(train_idx[:180032*9])
  #print(train_idx)
  test_df = train_df.tail(180032)
  return loss_weight,y_aux,y,train_idx,test_df
loss_weight,y_aux,y,train_idx,test_df = loadeverything()

def custom_loss(y_true, y_pred):
  return binary_crossentropy(K.reshape(y_true[:,0],(-1,1)),y_pred) * y_true[:,1]

In [10]:
from tensorflow.python import keras
#import keras.backend as K
import re
import codecs
def createmodel():
  adamwarm = AdamWarmup(lr=lr,decay_steps = decay_steps, warmup_steps = warmup_steps,kernel_weight_decay = weight_decay)
  sequence_output  = model.get_layer('Extract').output
  pool_output = keras.layers.Dense(units=1, activation='sigmoid',name='real_output')(sequence_output)
  aux_output = keras.layers.Dense(units=6, activation='sigmoid',name='aux_output')(sequence_output)
  model3  = keras.models.Model(inputs=model.input, outputs=[pool_output,aux_output])
  model3.compile(loss=[custom_loss,'binary_crossentropy'],loss_weights=[loss_weight,6.],optimizer=adamwarm)
  model3.summary()
  return model3
model3 = createmodel()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 220)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 220)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 220, 1024),  31254528    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 220, 1024)    2048        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

In [11]:
# @title Initialize Variables
import tensorflow as tf
import tensorflow.keras.backend as K
from keras_bert import get_custom_objects

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)


tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
)

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    model3 = tf.contrib.tpu.keras_to_tpu_model(model3, strategy=strategy)

INFO:tensorflow:Querying Tensorflow master (grpc://10.30.143.210:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 1794948670499916486)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 16711173968620041571)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 18096171597325454457)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 16585903583717639435)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 9160151419783854817)
INFO:tensorflow:*** Available Device: _DeviceAttribute

## Prepare Data, Training, Predicting

First the model need train data like [token_input,seg_input,masked input], here we set all segment input to 0 and all masked input to 1.

Still I am finding a more efficient way to do token-convert-to-ids

In [12]:
#print('sample used',train_df.shape[0])
#token_input = (np.load('new7_train_input_220.npy')[train_idx])
def runep1():
  token_input = (np.load('new7_train_input_220.npy')[train_idx])#[:16000]
  mask_input = np.ones((180032,maxlen))
  token_input2 = token_input[-180032:]
  np.save('token_input2',token_input2)
  #mask_input = np.ones((16000,maxlen))
  print(token_input.shape)
  #print(seg_input.shape)
  print(mask_input.shape)
  print('begin training')
  with tf.keras.utils.custom_object_scope(get_custom_objects()):
    for i in range(9):
    #for i in range(1):
      seg_input = np.zeros((180032,maxlen))
      #seg_input = np.zeros((16000,maxlen))
      #seg_input[(token_input[(180032*i):(180032*(i+1))])!=0]=1
      model3.fit([token_input[(180032*i):(180032*(i+1))], seg_input, mask_input],[y[(180032*i):(180032*(i+1))],
                                                                                  y_aux[(180032*i):(180032*(i+1))]],batch_size=bsz,epochs=nb_epochs)
      #model3.fit([token_input[(16000*i):(16000*(i+1))], seg_input, mask_input],[y[(16000*i):(16000*(i+1))],
      #                                                                           y_aux[(16000*i):(16000*(i+1))]],batch_size=bsz,epochs=nb_epochs)
  return model3
model3 = runep1()

(1804874, 220)
(180032, 220)
begin training
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(8,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(8, 220), dtype=tf.float32, name='Input-Token_10'), TensorSpec(shape=(8, 220), dtype=tf.float32, name='Input-Segment_10'), TensorSpec(shape=(8, 220), dtype=tf.float32, name='Input-Masked_10'), TensorSpec(shape=(8, 2), dtype=tf.float32, name='real_output_target_30'), TensorSpec(shape=(8, 6), dtype=tf.float32, name='aux_output_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning AdamWarmup {'decay_steps': 33841.0, 'warmup_steps': 1692.0, 'min_lr': 0.0, 'lr': 2.9999999242136255e-05, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'epsilon': 1e-07, 'kernel_weight_decay': 0.009999999776482582, 'bias_weight_decay': 0.0, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for Input-Token
INFO:tensorflow:Remapping placeholder for Input-Segment
INFO:tenso

In [13]:
#valid test data

print('valid data done')
def run_pred1():
  #token_input = (np.load('new7_train_input_220.npy')[train_idx])
  token_input2 = np.load('token_input2.npy')#token_input[-180032:]
  with tf.keras.utils.custom_object_scope(get_custom_objects()):
    seg_input = np.zeros((180032,maxlen))
    mask_input = np.ones((180032,maxlen))
    #seg_input[token_input2!=0]=1
    hehe = model3.predict([token_input2, seg_input, mask_input],verbose=1,batch_size=bsz)
  return hehe
hehe = run_pred1()

valid data done
INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(8, 220), dtype=tf.float32, name='Input-Token_10'), TensorSpec(shape=(8, 220), dtype=tf.float32, name='Input-Segment_10'), TensorSpec(shape=(8, 220), dtype=tf.float32, name='Input-Masked_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning AdamWarmup {'decay_steps': 33841.0, 'warmup_steps': 1692.0, 'min_lr': 0.0, 'lr': 2.9999999242136255e-05, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'epsilon': 1e-07, 'kernel_weight_decay': 0.009999999776482582, 'bias_weight_decay': 0.0, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for Input-Token
INFO:tensorflow:Remapping placeholder for Input-Segment
INFO:tensorflow:Remapping placeholder for Input-Masked
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 87.43157434463501 secs
180032/180032 [==============================] - 502s 3ms/sample


In [0]:
# From baseline kernel
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn import metrics
def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>0.5]
    return compute_auc((subgroup_examples[label]>0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>0.5) & (df[label]<=0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<=0.5) & (df[label]>0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>0.5) & (df[label]>0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<=0.5) & (df[label]<=0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)

In [15]:
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
MODEL_NAME = 'model1'
test_df[MODEL_NAME]=hehe[0].flatten()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.963948,0.904455,black,0.886296,1357
7,0.964249,0.916736,white,0.897247,2340
2,0.957101,0.928201,homosexual_gay_or_lesbian,0.904812,1021
5,0.958783,0.938699,muslim,0.918490,1929
0,0.963351,0.955292,male,0.948432,3966
4,0.956987,0.962144,jewish,0.951142,714
1,0.960643,0.962750,female,0.952999,5100
3,0.952749,0.971893,christian,0.959475,3511
8,0.973574,0.950253,psychiatric_or_mental_illness,0.960511,393


In [16]:
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

0.952892698891049

In [17]:
model3.save('ep1.h5')

INFO:tensorflow:Copying TPU weights to the CPU
INFO:tensorflow:TPU -> CPU decay_steps: 33841.0
INFO:tensorflow:TPU -> CPU warmup_steps: 1692.0
INFO:tensorflow:TPU -> CPU min_lr: 0.0
INFO:tensorflow:TPU -> CPU lr: 2.9999999242136255e-05
INFO:tensorflow:TPU -> CPU beta_1: 0.8999999761581421
INFO:tensorflow:TPU -> CPU beta_2: 0.9990000128746033
INFO:tensorflow:TPU -> CPU epsilon: 1e-07
INFO:tensorflow:TPU -> CPU kernel_weight_decay: 0.009999999776482582
INFO:tensorflow:TPU -> CPU bias_weight_decay: 0.0
INFO:tensorflow:TPU -> CPU amsgrad: False


In [18]:
ep2idx = np.arange(180032*9)
np.random.shuffle(ep2idx)
def run1p5():
  token_input = (np.load('new7_train_input_220.npy')[train_idx])
  with tf.keras.utils.custom_object_scope(get_custom_objects()):
    K.set_value(model3.optimizer.lr, 1e-5)
    mask_input = np.ones((180032,maxlen))
    for i in range(5):
      seg_input = np.zeros((180032,maxlen))
      #seg_input[(token_input[ep2idx[(180032*i):(180032*(i+1))]!=0]=1
      model3.fit([token_input[ep2idx[(180032*i):(180032*(i+1))]], seg_input, mask_input],[y[ep2idx[(180032*i):(180032*(i+1))]],
                                                                                  y_aux[ep2idx[(180032*i):(180032*(i+1))]]],batch_size=bsz,epochs=nb_epochs)
  return model3
model3 = run1p5()

180032/180032 [==============================] - 1431s 8ms/sample - loss: 0.6637 - real_output_loss: 0.0775 - aux_output_loss: 0.0775


In [19]:
def run_pred1p5():
  #token_input = (np.load('new7_train_input_220.npy')[train_idx])  
  #token_input2 = token_input[-180032:]
  token_input2 = np.load('token_input2.npy')
  with tf.keras.utils.custom_object_scope(get_custom_objects()):
    mask_input =  np.ones((180032,maxlen))
    seg_input = np.zeros((180032,maxlen))
    #seg_input[token_input2!=0]=1
    hehe = model3.predict([token_input2, seg_input, mask_input],verbose=1,batch_size=bsz)
  return hehe
hehe = run_pred1p5()
test_df[MODEL_NAME]=hehe[0].flatten()
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df

180032/180032 [==============================] - 354s 2ms/sample


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.965467,0.907034,black,0.891643,1357
7,0.963930,0.922880,white,0.901203,2340
2,0.966191,0.913683,homosexual_gay_or_lesbian,0.906153,1021
5,0.963740,0.933834,muslim,0.920936,1929
4,0.960001,0.957616,jewish,0.947178,714
0,0.964762,0.956409,male,0.950443,3966
1,0.961387,0.963063,female,0.953070,5100
8,0.977485,0.943386,psychiatric_or_mental_illness,0.959760,393
3,0.957337,0.970830,christian,0.961549,3511


In [20]:
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

0.9536535785034861

In [21]:
model3.save('ep1p5.h5')

INFO:tensorflow:Copying TPU weights to the CPU
INFO:tensorflow:TPU -> CPU decay_steps: 33841.0
INFO:tensorflow:TPU -> CPU warmup_steps: 1692.0
INFO:tensorflow:TPU -> CPU min_lr: 0.0
INFO:tensorflow:TPU -> CPU lr: 9.999999747378752e-06
INFO:tensorflow:TPU -> CPU beta_1: 0.8999999761581421
INFO:tensorflow:TPU -> CPU beta_2: 0.9990000128746033
INFO:tensorflow:TPU -> CPU epsilon: 1e-07
INFO:tensorflow:TPU -> CPU kernel_weight_decay: 0.009999999776482582
INFO:tensorflow:TPU -> CPU bias_weight_decay: 0.0
INFO:tensorflow:TPU -> CPU amsgrad: False


In [22]:
def runep2():
  token_input = (np.load('new7_train_input_220.npy')[train_idx])  
  with tf.keras.utils.custom_object_scope(get_custom_objects()):
    for i in range(5,9):
      mask_input =  np.ones((180032,maxlen))
      seg_input = np.zeros((180032,maxlen))
      model3.fit([token_input[ep2idx[(180032*i):(180032*(i+1))]], seg_input, mask_input],[y[ep2idx[(180032*i):(180032*(i+1))]],
                                                                                  y_aux[ep2idx[(180032*i):(180032*(i+1))]]],batch_size=bsz,epochs=nb_epochs)
  return model3
model3 = runep2()

180032/180032 [==============================] - 1424s 8ms/sample - loss: 0.6654 - real_output_loss: 0.0776 - aux_output_loss: 0.0776


In [23]:
def run_pred2():
  #token_input = (np.load('new7_train_input_220.npy')[train_idx])  
  token_input2 = np.load('token_input2.npy')
  with tf.keras.utils.custom_object_scope(get_custom_objects()):
    mask_input =  np.ones((180032,maxlen))
    seg_input = np.zeros((180032,maxlen))
    hehe = model3.predict([token_input2, seg_input, mask_input],verbose=1,batch_size=bsz)
  return hehe
hehe = run_pred2()
test_df[MODEL_NAME]=hehe[0].flatten()
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df

180032/180032 [==============================] - 354s 2ms/sample


,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
6,0.965467,0.907034,black,0.891643,1357
7,0.963930,0.922880,white,0.901203,2340
2,0.966191,0.913683,homosexual_gay_or_lesbian,0.906153,1021
5,0.963740,0.933834,muslim,0.920936,1929
4,0.960001,0.957616,jewish,0.947178,714
0,0.964762,0.956409,male,0.950443,3966
1,0.961387,0.963063,female,0.953070,5100
8,0.977485,0.943386,psychiatric_or_mental_illness,0.959760,393
3,0.957337,0.970830,christian,0.961549,3511


In [24]:
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

0.9536535785034861

In [25]:
!unzip predict_bertpre_220.zip.zip

Archive:  predict_bertpre_220.zip.zip
  inflating: bertpredict.npy         


In [26]:
#valid test data
#token_input2 = np.load('predict_gpt2raw260.npy')
#token_input2 = np.load('predict_gpt2pre2_260.npy')
#token_input2 = np.load('predict_gpt2raw260mask.npy')
def testep2():
  token_input2 = np.load('bertpredict.npy')
  seg_input = np.zeros((token_input2.shape[0],maxlen))
  mask_input = np.ones((token_input2.shape[0],maxlen))
  print('valid data done')
  print(token_input2.shape)
  print(seg_input.shape)
  print(mask_input.shape)
  with tf.keras.utils.custom_object_scope(get_custom_objects()):
    hehe = model3.predict([token_input2, seg_input, mask_input],verbose=1,batch_size=120)
  return hehe
hehe = testep2()

valid data done
(97320, 220)
(97320, 220)
(97320, 220)
INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(15, 220), dtype=tf.float32, name='Input-Token_10'), TensorSpec(shape=(15, 220), dtype=tf.float32, name='Input-Segment_10'), TensorSpec(shape=(15, 220), dtype=tf.float32, name='Input-Masked_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for Input-Token
INFO:tensorflow:Remapping placeholder for Input-Segment
INFO:tensorflow:Remapping placeholder for Input-Masked
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 64.12428975105286 secs
97320/97320 [==============================] - 299s 3ms/sample


In [0]:
np.save('bert_pre_220_94929',hehe[0])

In [28]:
model3.save('9513xxep2bertlarge.h5')

INFO:tensorflow:Copying TPU weights to the CPU
INFO:tensorflow:TPU -> CPU decay_steps: 33841.0
INFO:tensorflow:TPU -> CPU warmup_steps: 1692.0
INFO:tensorflow:TPU -> CPU min_lr: 0.0
INFO:tensorflow:TPU -> CPU lr: 9.999999747378752e-06
INFO:tensorflow:TPU -> CPU beta_1: 0.8999999761581421
INFO:tensorflow:TPU -> CPU beta_2: 0.9990000128746033
INFO:tensorflow:TPU -> CPU epsilon: 1e-07
INFO:tensorflow:TPU -> CPU kernel_weight_decay: 0.009999999776482582
INFO:tensorflow:TPU -> CPU bias_weight_decay: 0.0
INFO:tensorflow:TPU -> CPU amsgrad: False


In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
file_path_1 = '9513xxep2bertlarge.h5'
file1 = drive.CreateFile()
file1.SetContentFile(file_path_1)
file1.Upload()